In [1]:
#task 2 - Superstore_USA
import pandas as pd
import mysql.connector as connection
import numpy as np

In [2]:
# Connect to My sql Server
connect=connection.connect(host='localhost', user='root', password='Boost123')

In [3]:
# create cursor
cursor= connect.cursor()

In [4]:
# use ineuron DB
cursor.execute('use ineuron')

In [5]:
# 1 . load this data in sql and in pandas with a relation in sql 

In [6]:
# will use  openpyxl  to read the data
#!pip install  openpyxl 

In [7]:
import openpyxl
 
# Give the location of the file
wb_obj = openpyxl.load_workbook('Superstore_USA.xlsx') 

In [8]:
class Mysql_Operations:
    def __init__(self, table_name, dataset):
        self.table_name = table_name
        self.dataset = dataset
        self.max_col = dataset.max_column
        self.max_row= dataset.max_row


    def create_table(self):
        query = f'create table {self.table_name} ('
        
        # Loop will print all columns name
        for i in range(1, self.max_col + 1): 
            cell_obj1 = self.dataset.cell(row = 1, column = i)
            cell_obj2 = self.dataset.cell(row = 2, column = i)
            if (cell_obj2.data_type == 'n') and ('.' in str(cell_obj2.value)):
                query += "`" + '_'.join(cell_obj1.value.split(' ')) + "`" + ' float,'     
            elif cell_obj2.data_type == 's':
                query += "`" + '_'.join(cell_obj1.value.split(' ')) + "`" + ' varchar(100),'
            elif cell_obj2.data_type == 'd':
                query += "`" + '_'.join(cell_obj1.value.split(' ')) + "`" + ' datetime,'
            else:
                query += "`" + '_'.join(cell_obj1.value.split(' ')) + "`" + ' int,'
        else:
            query = query[0:len(query) - 1] + ')'

        try:
            cursor.execute(query)
            cursor.execute('show tables')
            cursor.fetchall()
            print(f'Table {self.table_name} created successfully')
        except Exception as e:
            print(e)

    def insert_data(self):
        try:
            col_count = ['%s' for i in range(self.max_col)]
            query = f"insert into {self.table_name} values({', '.join(col_count)})"
            for i in range(2, self.max_row + 1):
                rows=[self.dataset.cell(row = i, column = j).value for j in range(1, self.max_col+ 1)]
                cursor.execute(query, tuple(rows))
            connect.commit()
            print(f'Data inserted Successfully to {self.table_name}')
        except Exception as e:
            print(e)

In [ ]:
# 2 . while loading this data you dont have to create a table manually you can use any automated approach to create a table and load a data in bulk in table 

In [9]:
for i in wb_obj.sheetnames:
    print(wb_obj[i])
    supermarket=Mysql_Operations(i, wb_obj[i])
    print(wb_obj[i].max_row)
    supermarket.create_table()
    supermarket.insert_data()

<Worksheet "Orders">
9427
Table Orders created successfully
Data inserted Successfully to Orders
<Worksheet "Returns">
1635
Table Returns created successfully
Data inserted Successfully to Returns
<Worksheet "Users">
5
Table Users created successfully
Data inserted Successfully to Users


In [14]:
supermarket_data= pd.read_sql_query('Select * from Orders', connect)
supermarket_data.head()

/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Row_ID,Order_Priority,Discount,Unit_Price,Shipping_Cost,Customer_ID,Customer_Name,Ship_Mode,Customer_Segment,Product_Category,...,Region,State_or_Province,City,Postal_Code,Order_Date,Ship_Date,Profit,Quantity_ordered_new,Sales,Order_ID
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.320,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.560,4,13.01,88522
2,23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-47.640,7,49.92,88523
3,23087,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-30.510,7,41.64,88523
4,23088,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,Technology,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-27,998.202,8,1446.67,88523


In [16]:
returns_data= pd.read_sql_query('Select * from Returns', connect)
returns_data.head()

/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Order_ID,Status
0,65,Returned
1,612,Returned
2,614,Returned
3,678,Returned
4,710,Returned


In [19]:
User_data=pd.read_sql_query('Select * from Users', connect)
User_data.head()

/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Region,Manager
0,Central,Chris
1,East,Erin
2,South,Sam
3,West,William


In [20]:
# 4 . try  to join order and return data both in sql and pandas 
return_orders= pd.merge(returns_data, supermarket_data, on ='Order_ID')

In [21]:
# sql
query= '''select * from returns d inner join Orders a
on a.[Order_ID]=a.[Order_ID]'''

In [22]:
# 3 . Find out how return that we ahve recived and with a product id 
return_orders[['Product_Name','Ship_Mode']]

,Product_Name,Ship_Mode
0,Xerox 1928,Regular Air
1,6190,Regular Air
2,"80 Minute Slim Jewel Case CD-R , 10/Pack - Sta...",Regular Air
3,"#10 White Business Envelopes,4 1/8 x 9 1/2",Regular Air
4,Hon 2090 “Pillow Soft” Series Mid Back Swivel/...,Delivery Truck
...,...,...
93,Chromcraft Bull-Nose Wood Round Conference Tab...,Delivery Truck
94,Personal Creations™ Ink Jet Cards and Labels,Regular Air
95,Kleencut® Forged Office Shears by Acme United ...,Regular Air
96,Xerox 197,Express Air


In [23]:
# 5 . Try to find out how many unique customer that we have 
supermarket_data['Customer_Name'].nunique()

2703

In [24]:
# 6 . try to find out in how many regions we are selling a product and who is a manager for a respective region 
manager_data= pd.merge(supermarket_data,User_data, on ='Region')
manager_data.groupby('Region')['Manager'].unique()

Region
Central      [Chris]
East          [Erin]
South          [Sam]
West       [William]
Name: Manager, dtype: object

In [25]:
# 7 . find out how many different differnet shipement mode that we have and
#what is a percentage usablity of all the shipment mode with respect to dataset
supermarket_data['Ship_Mode'].value_counts(normalize=True)*100

Regular Air       74.644600
Delivery Truck    13.611288
Express Air       11.744112
Name: Ship_Mode, dtype: float64

In [26]:
# 8 . Create a new coulmn and try to find our a diffrence between order date and shipment date
supermarket_data['delivery']= supermarket_data['Ship_Date']-supermarket_data['Order_Date'] 
supermarket_data.head()

,Row_ID,Order_Priority,Discount,Unit_Price,Shipping_Cost,Customer_ID,Customer_Name,Ship_Mode,Customer_Segment,Product_Category,...,State_or_Province,City,Postal_Code,Order_Date,Ship_Date,Profit,Quantity_ordered_new,Sales,Order_ID,delivery
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Illinois,Addison,60101,2012-05-28,2012-05-30,1.320,2,5.90,88525,2 days
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.560,4,13.01,88522,1 days
2,23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,Washington,Anacortes,98221,2011-07-27,2011-07-28,-47.640,7,49.92,88523,1 days
3,23087,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,Office Supplies,...,Washington,Anacortes,98221,2011-07-27,2011-07-28,-30.510,7,41.64,88523,1 days
4,23088,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,Technology,...,Washington,Anacortes,98221,2011-07-27,2011-07-27,998.202,8,1446.67,88523,0 days


In [27]:
# 9 . base on question number 8 find out for which order id we have shipment duration more than 10 days 

supermarket_data['delivery'] = supermarket_data['delivery'].dt.days.astype('int16')
supermarket_data[supermarket_data['delivery']>10]

,Row_ID,Order_Priority,Discount,Unit_Price,Shipping_Cost,Customer_ID,Customer_Name,Ship_Mode,Customer_Segment,Product_Category,...,State_or_Province,City,Postal_Code,Order_Date,Ship_Date,Profit,Quantity_ordered_new,Sales,Order_ID,delivery
643,18246,Critical,0.05,11.09,5.25,250,Brenda Nelson Blanchard,Regular Air,Corporate,Office Supplies,...,Minnesota,Richfield,55423,2011-12-29,2012-03-22,58.500,19,204.67,87215,84
1548,23091,Medium,0.04,100.98,7.18,595,Dana Desai,Regular Air,Small Business,Technology,...,Indiana,Bloomington,47401,2011-12-30,2012-01-10,375.705,6,587.46,86318,11
1549,23092,Medium,0.09,6.48,7.49,595,Dana Desai,Express Air,Small Business,Office Supplies,...,Indiana,Bloomington,47401,2011-12-30,2012-01-14,-47.395,12,79.37,86318,15
1678,20703,Not Specified,0.07,130.98,30.00,637,Christopher Bryant,Delivery Truck,Consumer,Furniture,...,California,Santa Clara,95051,2011-12-30,2012-01-16,-211.885,21,2699.94,87957,17
1679,20704,Not Specified,0.09,55.98,4.86,637,Christopher Bryant,Regular Air,Consumer,Office Supplies,...,California,Santa Clara,95051,2011-12-30,2012-01-10,355.012,10,514.51,87957,11
1680,20705,Not Specified,0.08,115.99,4.23,637,Christopher Bryant,Regular Air,Consumer,Technology,...,California,Santa Clara,95051,2011-12-30,2012-01-27,1021.400,16,1480.29,87957,28
1697,2703,Not Specified,0.07,130.98,30.00,640,Neal Wolfe,Delivery Truck,Consumer,Furniture,...,Washington,Seattle,98119,2011-12-30,2012-01-16,-423.770,83,10671.20,19556,17
1698,2704,Not Specified,0.09,55.98,4.86,640,Neal Wolfe,Regular Air,Consumer,Office Supplies,...,Washington,Seattle,98119,2011-12-30,2012-01-10,469.690,41,2109.51,19556,11
1699,2705,Not Specified,0.08,115.99,4.23,640,Neal Wolfe,Regular Air,Consumer,Technology,...,Washington,Seattle,98119,2011-12-30,2012-01-27,727.362,65,6013.69,19556,28
2515,18060,Low,0.02,2.21,1.00,964,Virginia Rivera,Regular Air,Home Office,Office Supplies,...,California,Redwood City,94061,2011-12-30,2012-03-31,5.820,10,22.35,86177,92


In [28]:
# 10 . Try to find out a list of a returned order which sihpment duration was more then 15 days 
# and find out that region manager as well 

return_orders['delivery']= return_orders['Ship_Date']-return_orders['Order_Date'] 
return_orders['delivery'] = return_orders['delivery'].dt.days.astype('int16')

In [29]:
return_orders['delivery'].value_counts()

2    45
1    29
0    10
5     5
4     3
3     3
7     2
9     1
Name: delivery, dtype: int64

In [30]:
# there are no orders where the shipment duration was more than 15 days. so reframing to 9days
return_manager= pd.merge(return_orders, User_data, on= 'Region', how='left')
return_manager.head()

,Order_ID,Status,Row_ID,Order_Priority,Discount,Unit_Price,Shipping_Cost,Customer_ID,Customer_Name,Ship_Mode,...,State_or_Province,City,Postal_Code,Order_Date,Ship_Date,Profit,Quantity_ordered_new,Sales,delivery,Manager
0,612,Returned,95,Medium,0.10,5.28,6.26,3011,Tammy Raynor,Regular Air,...,Massachusetts,Boston,2113,2013-10-20,2013-10-22,-166.290,105,552.03,2,Erin
1,612,Returned,96,Medium,0.01,65.99,2.50,3011,Tammy Raynor,Regular Air,...,Massachusetts,Boston,2113,2013-10-20,2013-10-22,881.676,90,5297.63,2,Erin
2,4610,Returned,653,Low,0.09,8.33,1.99,2548,Wayne Bass,Regular Air,...,California,Los Angeles,90068,2011-06-17,2011-06-19,-17.010,12,100.32,2,William
3,4610,Returned,654,Low,0.08,15.67,1.39,2548,Wayne Bass,Regular Air,...,California,Los Angeles,90068,2011-06-17,2011-06-21,203.010,44,678.72,4,William
4,7079,Returned,979,Medium,0.01,280.98,57.00,478,Michael Wiggins,Delivery Truck,...,New York,New York City,10177,2011-02-08,2011-02-09,-416.700,44,12296.50,1,Erin


In [31]:
return_manager[return_manager['delivery']>8]

,Order_ID,Status,Row_ID,Order_Priority,Discount,Unit_Price,Shipping_Cost,Customer_ID,Customer_Name,Ship_Mode,...,State_or_Province,City,Postal_Code,Order_Date,Ship_Date,Profit,Quantity_ordered_new,Sales,delivery,Manager
80,51559,Returned,7234,Low,0.09,25.98,4.08,949,Ernest Oh,Regular Air,...,California,Los Angeles,90049,2013-03-04,2013-03-13,129.81,34,823.98,9,William


In [32]:
# 11 . Gorup by region and find out which region is more profitable 
supermarket_data.groupby('Region')['Profit'].sum().sort_values(ascending=False).index[0]

'Central'

In [33]:
# 12 . Try to find out overalll in which country we are giving more didscount 
supermarket_data.groupby('State_or_Province')['Discount'].mean().sort_values(ascending=False)[:10]

State_or_Province
Wyoming         0.054762
Louisiana       0.054607
Rhode Island    0.054500
Utah            0.052534
Minnesota       0.052385
Vermont         0.052295
Connecticut     0.052195
Maine           0.052031
Kansas          0.051805
Tennessee       0.051506
Name: Discount, dtype: float64

In [34]:
# 13 . Give me a list of unique postal code 
for codes in supermarket_data['Postal_Code'].unique():
    print(codes)

60101
98221
91776
95123
2724
3110
8101
8109
7203
2907
55372
11787
13210
59405
59601
59801
68005
10012
92653
92677
92530
92630
90712
93534
90260
97405
97526
97030
97123
97303
62002
98373
98052
75019
23834
62701
60107
60477
14150
12180
77642
75080
77471
78664
75088
24153
94043
92563
94559
10177
2917
5401
4070
2331
8830
8701
7470
7481
5201
97035
44708
45231
44105
95687
5451
94591
70056
22102
60601
66209
66215
66502
66062
6510
4401
4005
2129
2038
1852
1752
2067
3820
8618
7644
7110
8861
7095
2895
97128
97504
98103
84118
84041
93277
76903
78207
60004
94952
1510
1106
1880
6401
6360
2664
15122
76148
5439
5403
37664
37918
37086
37087
80525
80817
60452
76131
98059
99352
37804
70802
4092
7024
76240
98158
78415
94122
4011
4240
2474
2540
1580
3038
60505
84043
84321
66212
48138
85335
74006
75217
77536
90660
13501
11580
63114
84044
84047
84107
60453
2021
2766
7101
28227
60302
60462
50208
80027
66203
81503
80634
37922
38109
55423
55901
55113
80126
80232
17331
33772
87505
12208
44106
43229
78589
86001


In [35]:
# 14 . which customer segement is more profitalble find it out . 
supermarket_data.groupby('Customer_Segment')['Profit'].sum().sort_values(ascending=False).index[0]


'Corporate'

In [37]:
# 15 . try to find out the 10th most loss making product catagory . 
# supermarket_data.groupby('Product_Category')['Profit'].sum().sort_values().index[10]


#There are only 3 product Categries, identifying the 2nd 
supermarket_data.groupby('Product_Category')['Profit'].sum().sort_values().index[1]


'Office Supplies'

In [38]:
# 16 . Try to find out 10 top  product with highest margins 
supermarket_data.groupby('Product_Name')['Product_Base_Margin'].mean().sort_values(ascending=False)[:10]

Product_Name
Office Impressions Heavy Duty Welded Shelving & Multimedia Storage Drawers    0.85
Accessory27                                                                   0.85
Sterling Rubber Bands by Alliance                                             0.85
Belkin OmniView SE Rackmount Kit                                              0.85
Accessory4                                                                    0.85
Tennsco Stur-D-Stor Boltless Shelving, 5 Shelves, 24" Deep, Sand              0.84
Tennsco Commercial Shelving                                                   0.84
Serrated Blade or Curved Handle Hand Letter Openers                           0.84
Accessory36                                                                   0.83
Eldon Shelf Savers™ Cubes and Bins                                            0.83
Name: Product_Base_Margin, dtype: float64